<a href="https://colab.research.google.com/github/SoIllEconomist/ds4b/blob/master/python_ds4b/01_exploration/02_data_transformation/02_data_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Transformation

## Introduction

Visualisation is an important tool for insight generation, but it is rare that you get the data in exactly the right form you need. Often you’ll need to create some new variables or summaries, or maybe you just want to rename the variables or reorder the observations in order to make the data a little easier to work with. You’ll learn how to do all that (and more!) in this chapter, which will teach you how to transform your data using the dplyr package and a new dataset on flights departing New York City in 2013.
### Prerequisites
In this chapter we’re going to focus on how to use the dplyr package, another core member of the tidyverse. We’ll illustrate the key ideas using data from the NYC Flight data, and use `seaborn` to help us understand the data.

In [0]:
import pandas as pd
import numpy as np

flights = pd.read_csv("flight_data.csv")

### NYC Flights Dataset

To explore the basic data manipulation with `pandas`. The U.S. Department of Transportation's (DOT) Bureau of Transportation Statistics tracks the on-time performance of domestic flights operated by large air carriers. Summary information on the number of on-time, delayed, canceled, and diverted flights is published in DOT's monthly Air Travel Consumer Report and in this dataset of 2015 flight delays and cancellations. The data comes from the [US Bureau of Transportation Statistics](https://www.kaggle.com/usdot/flight-delays#flights.csv).

You might notice that this data frame prints a differently from other data frames you might have used in the past: it only shows the first few rows and all the columns that fit on one screen. (To see the whole dataset, you can run `flights` which will open the dataset instead of `flights.head()`. 

In [4]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
Unnamed: 0        100000 non-null int64
year              100000 non-null int64
month             100000 non-null int64
day               100000 non-null int64
sched_dep_time    100000 non-null float64
dep_delay         98592 non-null float64
arr_time          98478 non-null float64
dept_time         98592 non-null object
dtypes: float64(3), int64(4), object(1)
memory usage: 6.1+ MB


You might have noticed that `.info()` prints a concise summary of a DataFrame.

This method prints information about a DataFrame including the index dtype and column dtypes, non-null values and memory usage. 

## Pandas Basics

In this chapter you are going to learn the key pandas methods and funcgtions that allow you to solve the vast majority of your data manipulation challenges:

1. Pick observations by their values.
1. Reorder the rows.
1. Pick variables by their names.
1. Create new variables with functions of existing variables.
1. Collapse many values down to a single summary.

These can all be used in conjunction with `groupby()` which changes the scope of each function from operating on the entire dataset to operating on it group-by-group. These six functions provide the verbs for a language of data manipulation.

### Query

`query()` allows you to subset observations based on their values. The first argument is the name of the data frame. The second and subsequent arguments are the expressions that filter the data frame. For example, we can select all flights on January 1st with:

In [105]:
flights.query("month ==1 & day == 1")

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,1/1/2013 5:00
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,1/1/2013 5:00
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,1/1/2013 5:00
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,1/1/2013 5:00
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,1/1/2013 6:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,2013,1,1,2356.0,2359,-3.0,425.0,437,-12.0,B6,727,N588JB,JFK,BQN,186.0,1576,23,59,1/1/2013 23:00
838,2013,1,1,NaN,1630,NaN,NaN,1815,NaN,EV,4308,N18120,EWR,RDU,NaN,416,16,30,1/1/2013 16:00
839,2013,1,1,NaN,1935,NaN,NaN,2240,NaN,AA,791,N3EHAA,LGA,DFW,NaN,1389,19,35,1/1/2013 19:00
840,2013,1,1,NaN,1500,NaN,NaN,1825,NaN,AA,1925,N3EVAA,LGA,MIA,NaN,1096,15,0,1/1/2013 15:00


When you run that line of code, pandas executes the querying operation and returns a new data frame. pandas functions never modify their inputs, so if you want to save the result, you’ll need to use the assignment operator, `=`:

In [0]:
jan1 = flights.query("month ==1 & day == 1")

### Comparisons

To use querying effectively, you have to know how to select the observations that you want using the comparison operators. Python provides the standard suite: `>`, `>=`, `<`, `<=`, `!=` (not equal), and `==` (equal).

When you’re starting out with Python, the easiest mistake to make is to use = instead of == when testing for equality. When this happens you’ll get an informative error:

In [104]:
flights.query("month =1")

ValueError: ignored

There’s another common problem you might encounter when using ==: floating point numbers. These results might surprise you!

In [0]:
from math import sqrt

In [9]:
sqrt(2) ** 2 == 2

False

In [10]:
1/49 * 49 == 1

False

Computers use finite precision arithmetic (they obviously can’t store an infinite number of digits!) so remember that every number you see is an approximation.

## Logical Operators

Multiple arguments to `query()` are combined with “and”: every expression must be true in order for a row to be included in the output. For other types of combinations, you’ll need to use Boolean operators yourself: `&` is “and”, `|` is “or”, and `~` is “not”. The figure below shows the complete set of Boolean operations.

![Complete set of boolean operations. x is the left-hand circle, y is the right-hand circle, and the shaded region show which parts each operator selects.](https://github.com/SoIllEconomist/ds4b/blob/master/python_ds4b/01_exploration/02_data_transformation/transform_logical.png?raw=1)



In [11]:
flights.query("month in [11, 12]")

,Unnamed: 0,year,month,day,sched_dep_time,dep_delay,arr_time,dept_time
5,5,2015,11,14,317.0,4.0,824.0,313.0
18,18,2015,12,6,125.0,53.0,2339.0,72.0
26,26,2015,12,24,157.0,-5.0,1107.0,162.0
29,29,2015,11,21,160.0,-10.0,1656.0,170.0
32,32,2015,11,14,66.0,-4.0,1300.0,70.0
...,...,...,...,...,...,...,...,...
99981,99981,2015,11,22,167.0,-5.0,1640.0,172
99990,99990,2015,11,15,90.0,13.0,1319.0,77
99997,99997,2015,11,8,90.0,-5.0,1908.0,95
99998,99998,2015,12,20,70.0,-4.0,1427.0,74


Sometimes you can simplify complicated subsetting by remembering De Morgan’s law: `!(x & y)` is the same as `!x | !y`, and `!(x | y)` is the same as `!x & !y`. For example, if you wanted to find flights that weren’t delayed (on arrival or departure) by more than two hours, you could use either of the following two filters:

In [101]:
flights.query("~(arr_delay > 120 | dep_delay > 120)")

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,1/1/2013 5:00
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,1/1/2013 5:00
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,1/1/2013 5:00
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,1/1/2013 5:00
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,1/1/2013 6:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177345,2013,4,13,2135.0,2140,-5.0,24.0,28,-4.0,B6,43,N561JB,JFK,MCO,138.0,944,21,40,13-04-2013 21:00
177347,2013,4,13,2137.0,2135,2.0,2249.0,2252,-3.0,EV,4421,N12924,EWR,BTV,43.0,266,21,35,13-04-2013 21:00
177348,2013,4,13,2140.0,2150,-10.0,22.0,40,-18.0,B6,515,N273JB,EWR,FLL,151.0,1065,21,50,13-04-2013 21:00
177349,2013,4,13,2147.0,2140,7.0,2251.0,2310,-19.0,UA,1066,N37466,EWR,BOS,43.0,200,21,40,13-04-2013 21:00


In [102]:
flights.query("arr_delay <= 120 | dep_delay <= 120") # Double Check

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,1/1/2013 5:00
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,1/1/2013 5:00
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,1/1/2013 5:00
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,1/1/2013 5:00
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,1/1/2013 6:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177345,2013,4,13,2135.0,2140,-5.0,24.0,28,-4.0,B6,43,N561JB,JFK,MCO,138.0,944,21,40,13-04-2013 21:00
177347,2013,4,13,2137.0,2135,2.0,2249.0,2252,-3.0,EV,4421,N12924,EWR,BTV,43.0,266,21,35,13-04-2013 21:00
177348,2013,4,13,2140.0,2150,-10.0,22.0,40,-18.0,B6,515,N273JB,EWR,FLL,151.0,1065,21,50,13-04-2013 21:00
177349,2013,4,13,2147.0,2140,7.0,2251.0,2310,-19.0,UA,1066,N37466,EWR,BOS,43.0,200,21,40,13-04-2013 21:00


Whenever you start using complicated, multipart expressions in query(), consider making them explicit variables instead. That makes it much easier to check your work. You’ll learn how to create new variables shortly.

### Missing Values

One important feature of R that can make comparison tricky are missing values, or `NA`s (“not availables”). `NA` represents an unknown value so missing values are “contagious”: almost any operation involving an unknown value will also be unknown or False.

In [25]:
np.nan

nan

In [26]:
np.nan > 5

False

In [27]:
np.nan < 5

False

In [28]:
np.nan + 10

nan

In [29]:
np.nan / 2

nan

The most confusing result is this one:

In [30]:
np.nan == np.nan

False

It’s easiest to understand why this is true with a bit more context:

In [32]:
# Let x be Mary's age. We don't know how old she is.
x = np.nan

# Let y be John's age. We don't know how old he is.
y = np.nan

# Are John and Mary the same age?
x == y
# We don't know!

False

If you want to determine if a value is missing, use `pd.isna()`

In [34]:
pd.isna(x)

True

`query()` only includes rows where the condition is TRUE; it excludes both FALSE.

### Exercises

1. Find all the flights that
  1. Had an arrival delay of two or more hours
  1. Flew to Houston (IAH or HOU)
  1. Were operated by United, American, or Delta
  1. Departed in summer (July, August, and September)
  1. Arrived more than two hours late, but didn’t leave late
  1. Were delayed by at least an hour, but made up over 30 minutes in flight
  1. Departed between midnight and 6am (inclusive)6am

1. How many flights have a missing dep_time? What other variables are missing? What might these rows represent?

## Arrange rows with `sort_values()`
`sort_values()` works similarly to `query()` except that instead of selecting rows, it changes their order. It takes a data frame and a set of column names (or more complicated expressions) to order by. If you provide more than one column name, each additional column will be used to break ties in the values of preceding columns:


In [103]:
flights.sort_values(by=["year", "month", "day"])

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,1/1/2013 5:00
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,1/1/2013 5:00
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,1/1/2013 5:00
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,1/1/2013 5:00
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,1/1/2013 6:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111291,2013,12,31,NaN,705,NaN,NaN,931,NaN,UA,1729,NaN,EWR,DEN,NaN,1605,7,5,31-12-2013 07:00
111292,2013,12,31,NaN,825,NaN,NaN,1029,NaN,US,1831,NaN,JFK,CLT,NaN,541,8,25,31-12-2013 08:00
111293,2013,12,31,NaN,1615,NaN,NaN,1800,NaN,MQ,3301,N844MQ,LGA,RDU,NaN,431,16,15,31-12-2013 16:00
111294,2013,12,31,NaN,600,NaN,NaN,735,NaN,UA,219,NaN,EWR,ORD,NaN,719,6,0,31-12-2013 06:00


Use `ascending=False` to re-order by a column in descending order:

In [116]:
flights.sort_values(by=["year", "month", "day"], ascending=False).head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
110520,2013,12,31,13.0,2359,14.0,439.0,437,2.0,B6,839,N566JB,JFK,BQN,189.0,1576,23,59,31-12-2013 23:00
110521,2013,12,31,18.0,2359,19.0,449.0,444,5.0,DL,412,N713TW,JFK,SJU,192.0,1598,23,59,31-12-2013 23:00
110522,2013,12,31,26.0,2245,101.0,129.0,2353,96.0,B6,108,N374JB,JFK,PWM,50.0,273,22,45,31-12-2013 22:00
110523,2013,12,31,459.0,500,-1.0,655.0,651,4.0,US,1895,N557UW,EWR,CLT,95.0,529,5,0,31-12-2013 05:00
110524,2013,12,31,514.0,515,-1.0,814.0,812,2.0,UA,700,N470UA,EWR,IAH,223.0,1400,5,15,31-12-2013 05:00


Missing values are sorted at the end:

In [115]:
flights.sort_values(by=["year", "month", "dep_time"]).tail()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
111291,2013,12,31,NaN,705,NaN,NaN,931,NaN,UA,1729,NaN,EWR,DEN,NaN,1605,7,5,31-12-2013 07:00
111292,2013,12,31,NaN,825,NaN,NaN,1029,NaN,US,1831,NaN,JFK,CLT,NaN,541,8,25,31-12-2013 08:00
111293,2013,12,31,NaN,1615,NaN,NaN,1800,NaN,MQ,3301,N844MQ,LGA,RDU,NaN,431,16,15,31-12-2013 16:00
111294,2013,12,31,NaN,600,NaN,NaN,735,NaN,UA,219,NaN,EWR,ORD,NaN,719,6,0,31-12-2013 06:00
111295,2013,12,31,NaN,830,NaN,NaN,1154,NaN,UA,443,NaN,JFK,LAX,NaN,2475,8,30,31-12-2013 08:00


Puttingi NAs first:

In [114]:
flights.sort_values(by=["year", "month", "dep_time"], na_position='first')

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
838,2013,1,1,NaN,1630,NaN,NaN,1815,NaN,EV,4308,N18120,EWR,RDU,NaN,416,16,30,1/1/2013 16:00
839,2013,1,1,NaN,1935,NaN,NaN,2240,NaN,AA,791,N3EHAA,LGA,DFW,NaN,1389,19,35,1/1/2013 19:00
840,2013,1,1,NaN,1500,NaN,NaN,1825,NaN,AA,1925,N3EVAA,LGA,MIA,NaN,1096,15,0,1/1/2013 15:00
841,2013,1,1,NaN,600,NaN,NaN,901,NaN,B6,125,N618JB,JFK,FLL,NaN,1069,6,0,1/1/2013 6:00
1777,2013,1,2,NaN,1540,NaN,NaN,1747,NaN,EV,4352,N10575,EWR,CVG,NaN,569,15,40,2/1/2013 15:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91492,2013,12,9,2400.0,2359,1.0,432.0,440,-8.0,B6,1503,N705JB,JFK,SJU,195.0,1598,23,59,9/12/2013 23:00
91493,2013,12,9,2400.0,2250,70.0,59.0,2356,63.0,B6,1816,N187JB,JFK,SYR,41.0,209,22,50,9/12/2013 22:00
95379,2013,12,13,2400.0,2359,1.0,432.0,440,-8.0,B6,1503,N587JB,JFK,SJU,192.0,1598,23,59,13-12-2013 23:00
100795,2013,12,19,2400.0,2359,1.0,434.0,440,-6.0,B6,1503,N561JB,JFK,SJU,193.0,1598,23,59,19-12-2013 23:00


### Exercises
1. How could you use `sort_values()` to sort all missing values to the start?.

1. Sort flights to find the most delayed flights. Find the flights that left earliest.

1. Sort flights to find the fastest flights.

1. Which flights travelled the longest? Which travelled the shortest?



## Select columns with `[]`, `loc`, and `filter()`

It’s not uncommon to get datasets with hundreds or even thousands of variables. In this case, the first challenge is often narrowing in on the variables you’re actually interested in. Bracket notation `[]` allows you to rapidly zoom in on a useful subset using operations based on the names of the variables.

Bracket notation is not terribly useful with the flights data because we only have 19 variables, but you can still get the general idea:

In [79]:
flights[['year','month','day']]

,year,month,day
0,2015,2,23
1,2015,4,1
2,2015,10,22
3,2015,7,13
4,2015,10,14
...,...,...,...
99995,2015,10,7
99996,2015,6,23
99997,2015,11,8
99998,2015,12,20


In [80]:
flights.filter(["year", "month", "day"])

,year,month,day
0,2015,2,23
1,2015,4,1
2,2015,10,22
3,2015,7,13
4,2015,10,14
...,...,...,...
99995,2015,10,7
99996,2015,6,23
99997,2015,11,8
99998,2015,12,20


In [76]:
flights.loc[:,'year':'day']

,year,month,day
0,2015,2,23
1,2015,4,1
2,2015,10,22
3,2015,7,13
4,2015,10,14
...,...,...,...
99995,2015,10,7
99996,2015,6,23
99997,2015,11,8
99998,2015,12,20


There are a number of helper functions you can use:

```python
df.filter(regex="^abc") # starts with abc
df.filter(regex="$xyz") # ends with xyz
df.filter(regex="example") # contains example
```

## rename

In [85]:
df = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})
df

,A,B
0,1,4
1,2,5
2,3,6


Rename columns using a mapping:

In [86]:
df.rename(columns={"A": "a", "B": "c"})

,a,c
0,1,4
1,2,5
2,3,6


Rename index using a mapping:

In [87]:
df.rename(index={0: "x", 1: "y", 2: "z"})

,A,B
x,1,4
y,2,5
z,3,6


Using axis-style parameters

In [88]:
df.rename(str.lower, axis='columns')

,a,b
0,1,4
1,2,5
2,3,6


In [89]:
df.rename({1: 2, 2: 4}, axis='index')

,A,B
0,1,4
2,2,5
4,3,6


If you want any changes to stick, you need to use the `inplace=True` arguments.

In [0]:
df.rename(str.lower, axis="columns", inplace=True)

In [117]:
df # Now we see that everything is changed.

,a,b
0,1,4
1,2,5
2,3,6


In [0]:
flights.rename({"tailnum":"tail_num"}, axis="columns", inplace=True)

In [124]:
flights.columns

Index(['year', 'month', 'day', 'dep_time', 'sched_dep_time', 'dep_delay',
       'arr_time', 'sched_arr_time', 'arr_delay', 'carrier', 'flight',
       'tail_num', 'origin', 'dest', 'air_time', 'distance', 'hour', 'minute',
       'time_hour'],
      dtype='object')

### Exercises
1. Brainstorm as many ways as possible to select dep_time, dep_delay, arr_time, and arr_delay from flights.

1. What happens if you include the name of a variable multiple times in a selection statement call?

1. What does the filter and loc methods do? Why might it be helpful in conjunction with this list?
```python
vars = ["year", "month", "day", "dep_delay", "arr_delay"]
```

## Add new variables

Besides selecting sets of existing columns, it’s often useful to add new columns that are functions of existing columns.

In [0]:
flights["gain"] = flights.dep_delay - flights.arr_delay
flights["speed"] = flights.distance / flights.arr_time * 60
flights['hours'] = flights.air_time / 60
flights["gain_per_hour"] = flights.gain / flights.hours

### Exercises
1. Currently dep_time and `sched_dep_time` are convenient to look at, but hard to compute with because they’re not really continuous numbers. Convert them to a more convenient representation of number of minutes since midnight.

1. Compare air_time with `arr_time` - `dep_time`. What do you expect to see? What do you see? What do you need to do to fix it?

1. Compare `dep_time`, `sched_dep_time`, and `dep_delay`. How would you expect those three numbers to be related?

1. Find the 10 most delayed flights using a ranking function. How do you want to handle ties?

## Grouped summaries